In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import torch.nn.functional as F
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    BitsAndBytesConfig,
    Trainer)

 
import torch
import numpy as np
from torch import nn
 

In [ ]:
dev=np.array([302, 307, 331, 335, 346, 367, 377, 381, 382, 388, 389, 390, 395,
       403, 404, 406, 413, 417, 418, 420, 422, 436, 439, 440, 451, 458,
       472, 476, 477, 482, 483, 484, 489, 490, 492])


trainn=np.array([303, 304, 305, 310, 312, 313, 315, 316, 317, 318, 319, 320, 321,
       322, 324, 325, 326, 327, 328, 330, 333, 336, 338, 339, 340, 341,
       343, 344, 345, 347, 348, 350, 351, 352, 353, 355, 356, 357, 358,
       360, 362, 363, 364, 366, 368, 369, 370, 371, 372, 374, 375, 376,
       379, 380, 383, 385, 386, 391, 392, 393, 397, 400, 401, 402, 409,
       412, 414, 415, 416, 419, 423, 425, 426, 427, 428, 429, 430, 433,
       434, 437, 441, 443, 444, 445, 446, 447, 448, 449, 454, 455, 456,
       457, 459, 463, 464, 468, 471, 473, 474, 475, 478, 479, 485, 486,
       487, 488, 491])

In [ ]:

import pickle

import torch

model_checkpoint='facebook/opt-350m' 
tokenizerr = AutoTokenizer.from_pretrained(  model_checkpoint,add_prefix_space=True)


def pad_sequences_pytorch(sequences, max_length=None, padding_value=0, padding='post'):
    """
    Pad sequences to the same length with PyTorch, with optional truncating.

    Args:
        sequences (list of list of int): List of sequences, where each sequence is a list of integers.
        max_length (int, optional): Maximum length of all sequences. If None, uses the length of the longest sequence. 
                                    Sequences longer than max_length will be truncated. Defaults to None.
        padding_value (int, optional): Value used for padding. Defaults to 0.
        padding (str, optional): Whether to pad 'pre' (before) or 'post' (after) the sequences. Defaults to 'post'.

    Returns:
        torch.Tensor: A tensor of shape (batch_size, max_sequence_length) containing the padded (and possibly truncated) sequences.
    """
    if max_length is None:
        max_length = max(len(seq) for seq in sequences)
    
    # Initialize an empty tensor for the padded sequences
    padded_sequences = torch.full((len(sequences), max_length), padding_value, dtype=torch.int64)
    
    # Fill the tensor with the sequences
    for i, seq in enumerate(sequences):
        seq_length = min(len(seq), max_length)
        seq_tensor = torch.tensor(seq[:seq_length], dtype=torch.int64)
        if padding == 'pre':
            # Pad at the beginning
            padded_sequences[i, -seq_length:] = seq_tensor
        else:
            # Pad at the end (default)
            padded_sequences[i, :seq_length] = seq_tensor
            
    return padded_sequences

# Example usage
sequences = [[1, 2, 3, 4, 5], [4, 5], [6, 7, 8, 9, 10, 11]]
padded_sequences = pad_sequences_pytorch(sequences, max_length=2, padding_value=0, padding='post')
 
file_path =  'depression7a.pkl'
 

with open(file_path, 'rb') as file:
    loaded_list = pickle.load(file)

we=[]
for i in range(len(loaded_list[0])):
    
 
    inputs = tokenizerr.encode(loaded_list[0][i], return_tensors="pt",padding=True)
    we.append(inputs[0].tolist())


max_length  = 2048#max(len(seq) for seq in sequences)
padded_sequences =  pad_sequences_pytorch(we, max_length=max_length, padding='post')
inputs = torch.tensor(padded_sequences)


from sklearn.model_selection import train_test_split
a,b,c,d,ee,f=train_test_split(torch.tensor(loaded_list[2]), inputs,torch.tensor(loaded_list[1]) ,test_size=0.2,stratify=torch.tensor(loaded_list[1]))

 

from torch.utils.data import TensorDataset, DataLoader


ss = trainn 
indices = [i for i, value in enumerate(loaded_list[3]) if value in ss]


a=torch.tensor(loaded_list[2])[indices]
c=inputs[indices]
ee=torch.tensor(loaded_list[1])[indices]



ss = dev 
indices = [i for i, value in enumerate(loaded_list[3]) if value in ss]


b=torch.tensor(loaded_list[2])[indices]
d=inputs[indices]
f=torch.tensor(loaded_list[1])[indices]



dataset = TensorDataset(torch.tensor(a),torch.tensor(c),torch.tensor(ee))
datasett = TensorDataset(b,d,f)
loader = DataLoader(dataset, batch_size=5 , shuffle=True)

In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
loss_fn = nn.CrossEntropyLoss()

In [ ]:
import whisper

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
import base64
import gzip
from dataclasses import dataclass
from typing import Dict, Iterable, Optional

import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor, nn

In [ ]:

class Conv1d(nn.Conv1d):
    
    def _conv_forward(
        self, x: Tensor, weight: Tensor, bias: Optional[Tensor]
    ) -> Tensor:
        return super()._conv_forward(
            x, weight.to(x.dtype), None if bias is None else bias.to(x.dtype)
        )

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dropout_rate=0.1):
        super(DecoderLayer, self).__init__()
        
        # Self attention for the query
        self.self_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, dropout=dropout_rate)
        
        # Multi-head attention for query and key/value from encoder (x)
        self.cross_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=nhead, dropout=dropout_rate)
        
        # Dropouts for post-attention
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # Feed-forward network
        self.ffn = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(4 * d_model, d_model)
        )
        
        # Another layer normalization for the feed-forward network
        self.norm3 = nn.LayerNorm(d_model)

    def forward(self, query, x, query_mask=None):
        # Self attention
        # print(query_mask.shape,"skjdjkflskjlfmsd,mf,sdsssss
        self_attn_output, _ =  self.self_attn(query, query, query, key_padding_mask=query_mask)
        query = self.norm1(query + self.dropout1(self_attn_output))
        
        # Cross attention
        # print(query.shape,x.shape)
        attn_output, _ = self.cross_attn(query, x, x)
        query = self.norm2(query + self.dropout2(attn_output))
        
        # Feed-forward network
        ffn_output = self.ffn(query)
        output = self.norm3(query + ffn_output)
        
        return output

class MultiLayerDecoder(nn.Module):
    def __init__(self, d_model, nhead, query_dim, num_layers, max_seq_length, dropout_rate=0.0, num_classes=8):
        super(MultiLayerDecoder, self).__init__()
        
        # Query transformation
#         self.query_transform = nn.Linear(query_dim, d_model)
        
        # Decoder layers
        self.layers = nn.ModuleList([DecoderLayer(d_model, nhead, dropout_rate) for _ in range(num_layers)])
        
        # Classification layer
        # self.classifier = nn.Linear(d_model,  8)

        # CLS token & position embedding
        self.cls_token = nn.Parameter(torch.randn(1,1,d_model))
        self.pos_embedding = nn.Parameter(torch.randn(1, max_seq_length+1, d_model))
        self.register_buffer("positional_embedding", sinusoids(max_seq_length+1, d_model))
    def forward(self, query, x ,query_lengths):
        batch_size = query.size(0)
        
        # Transform query
        
#         query = self.query_transform(query)

        # Add CLS token
        cls_tokens = self.cls_token.repeat(batch_size, 1, 1)
        query = torch.cat([cls_tokens, query], dim=1)
        # print(query.size(1) ,"kjlkjlk")
        # Add position embedding
        query += self.positional_embedding#self.pos_embedding#[:, :query.size(1), :]
        query = query.transpose(0, 1)  # Transposed for MHA compatibility
        
        # Adjust query_lengths for added CLS token
        query_lengths += 1
        
        # Generate the mask based on query lengths
        query_mask = create_mask(query_lengths, query.shape[0])
        
        # Pass query through each decoder layer
        
        for layer in self.layers:
            query = layer(query, x ,query_mask )
        
        # Using CLS token for classification
        cls_output = query[0]
        
        
        return cls_output

In [ ]:
def sinusoids(length, channels, max_timescale=10000):
    """Returns sinusoids for positional embedding"""
    assert channels % 2 == 0
    log_timescale_increment = np.log(max_timescale) / (channels // 2 - 1)
    inv_timescales = torch.exp(-log_timescale_increment * torch.arange(channels // 2))
    scaled_time = torch.arange(length)[:, np.newaxis] * inv_timescales[np.newaxis, :]
    return torch.cat([torch.sin(scaled_time), torch.cos(scaled_time)], dim=1)
 
  
def create_mask(seq_lengths, max_len):
    mask = torch.arange(max_len).expand(len(seq_lengths), max_len).to(seq_lengths.device)
    mask = mask >= seq_lengths.unsqueeze(-1)
    return mask

In [ ]:
class T(nn.Module):
    def __init__(self  ):
        super(T, self).__init__()
       
        model = whisper.load_model("base.en")
        #model=model.to("cuda:0")
        encoder=nn.DataParallel(model.encoder, device_ids = [0,1,2,3]) 
        encoder= encoder.to("cuda:0")
       
        self.encoder=  encoder #model.encoder.to("cuda:0")
        d_model =  512
        nhead = 16
        query_dim = 512#3072#4096#2048
        num_layers =   8
        self.transform = nn.Linear(d_model, d_model)
        self.decoder =   MultiLayerDecoder(d_model, nhead, query_dim, num_layers,2048)
        
        self.model =   AutoModelForCausalLM.from_pretrained( model_checkpoint ,pad_token_id=0,device_map="auto",trust_remote_code=True)

 
        self.conv1 =Conv1d(80,80, kernel_size=32,stride=30, padding=1)# Conv1d(n_mels,n_mels, kernel_size=7,stride=6, padding=1)
       
        


    def forward(self, x,au):
    
     
        x=self.encoder( self.conv1(x).to("cuda:0") )
        x=x.to("cpu")
        
        
        mask = (au != 0).int()
        valid_lengths = [(row == 0).nonzero()[0][0].item() if (row == 0).any() else row.size(0) for row in mask]
        
       
        
        with torch.no_grad():
            au =  self.model(au,mask )[0]
        
        
        
 
        x1= (x).transpose(0, 1)
     
        
        logits = self.decoder(au,x1,torch.tensor(valid_lengths) )
        out=self.transform(logits)
      
        return logits ,   out

In [ ]:
import torch
import numpy as np

def extract_features(model, data_loader ):
    model.eval()  # Set model to evaluation mode
    features = []
    labels = []

    with torch.no_grad():  # Turn off gradients to save memory and computations
        for data  in data_loader:
            target=data[2]
            data = data#.to(device)
            output,_ = model(data[0],data[1])  # Get the features from the model
            features.extend(output.cpu().numpy())  # Store features
            labels.extend(target.numpy())  # Store labels

    return np.array(features), np.array(labels)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

ss = trainn#np.array( df1['Participant_ID'].tolist())
indices = [i for i, value in enumerate(loaded_list[3]) if value in ss]


X_train_=torch.tensor(loaded_list[2])[indices]
X_train=inputs[indices]
y_train=torch.tensor(loaded_list[1])[indices] 


ss = dev#np.array( df1['Participant_ID'].tolist())
indices = [i for i, value in enumerate(loaded_list[3]) if value in ss]


X_test_=torch.tensor(loaded_list[2])[indices]
X_test=inputs[indices]
y_test=torch.tensor(loaded_list[1])[indices] 


train_data = TensorDataset(X_train_,X_train, y_train)
test_data = TensorDataset(X_test_,X_test, y_test)
 
batch_size = 12  # Adjust as per your system's capability
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score



def compute_metrics(conf_matrix):
    # Precision, Recall, and F1 for each class
    TP = torch.diag(conf_matrix)
    FP = conf_matrix.sum(0) - TP
    FN = conf_matrix.sum(1) - TP
    TN = conf_matrix.sum() - (TP + FP + FN)
    
    precision = TP / (TP + FP).clamp(min=1)  # Prevent division by zero
    recall = TP / (TP + FN).clamp(min=1)
    accuracy_per_class = (TP + TN) / conf_matrix.sum()  # Accuracy per class might not be typically reported
    f1_score = 2 * (precision * recall) / (precision + recall).clamp(min=1)
    
    # Overall accuracy
    overall_accuracy = TP.sum() / conf_matrix.sum()
    
    # Handle divisions by zero
    precision[precision != precision] = 0  # NaNs caused by zero division are set to 0
    recall[recall != recall] = 0
    f1_score[f1_score != f1_score] = 0
    
    return precision, recall, accuracy_per_class, f1_score, overall_accuracy
 


def evall():
 
    dropout=0.2
    class NeuralNet(nn.Module):

        def __init__(self):
            super(NeuralNet, self).__init__()
            self.fc1 = nn.Linear( 512, 1024)#512
            self.batch_norm1 = nn.BatchNorm1d(1024)
            self.dropout1 = nn.Dropout(dropout)
            self.fc2 = nn.Linear(1024, 512)
            self.batch_norm2 = nn.BatchNorm1d(512)
            self.dropout2 = nn.Dropout(dropout)
            self.fc3 = nn.Linear(span_length, 256)
            self.batch_norm3 = nn.BatchNorm1d(256)
            self.dropout3 = nn.Dropout(dropout)
            self.fc4 = nn.Linear(256, 128)
            self.batch_norm4 = nn.BatchNorm1d(128)
            self.dropout4 = nn.Dropout(dropout)
            self.fc5 = nn.Linear(128, 64)
            self.batch_norm5 = nn.BatchNorm1d(64)
            self.fc6 = nn.Linear(64, 2)

        def forward(self, x):
#             x = F.relu(self.batch_norm1(self.fc1(x)))
#             x = self.dropout1(x)
#             x = F.relu(self.batch_norm2(self.fc2(x)))
#             x = self.dropout2(x)
            x = F.relu(self.batch_norm3(self.fc3(x)))
            x = self.dropout3(x)
            x = F.relu(self.batch_norm4(self.fc4(x)))
            x = self.dropout4(x)
            x = F.relu(self.batch_norm5(self.fc5(x)))
            x = self.fc6(x)
            return x







    # Initialize the model, loss function, and optimizer
    
    
    ff=[]
    wr=[]
    for mm in range(300):
        
        
        
        total_length = 512
        span_length =400#random.randint(350, 512)#300# 400

        # Randomly choose a start position that allows a full span of 3000 to fit
        start_position = random.randint(0, total_length - span_length)

        # End position is start plus the span length
        end_position = start_position + span_length
        
        
        
        
        
        train_featuress,test_featuress=train_features[:,start_position:end_position],test_features[:,start_position:end_position]
        
        
        #process_batch(torch.tensor(train_features)).numpy(),process_batch(torch.tensor(test_features)).numpy()
        
        
        
        
        train_dataset = TensorDataset(torch.tensor(  train_featuress), torch.tensor(train_labels))

        #test_features, test_labels = extract_features(model,  test_loader, device)
        test_dataset = TensorDataset(torch.tensor(test_featuress), torch.tensor(test_labels))

        train_loaderr = DataLoader(train_dataset, batch_size=64, shuffle=True)
        test_loaderr = DataLoader(test_dataset, batch_size=64, shuffle=False)

    
    
    
    
    
        modell =NeuralNet() 
        modell=modell.to("cuda:1")



        modell.train()
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(modell.parameters(), lr=0.00001,weight_decay=0.2)

        # Training loop
        num_epochs  =200
        for epoch in range(num_epochs):
            for inputs, labels in train_loaderr:
                # Forward pass
                inputs, labels=inputs.to("cuda:1"),labels.to("cuda:1")
                outputs = modell(inputs)
                loss = criterion(outputs, labels)

                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()

                optimizer.step()

        #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

        # Evaluate the model
        modell.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in test_loaderr:
                inputs, labels=inputs.to("cuda:1"),labels.to("cuda:1")
                outputs = modell(inputs)
                probabilities = F.softmax(outputs, dim=-1)
                
                wr.append(probabilities)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        #print(accuracy)



        f1 = f1_score(labels.cpu(), predicted.cpu(), average='macro')

        #print(f'F1 score of kNN classifier on test set: {f1:.2f}')




        ff.append( predicted.cpu())

    
    stacked_predictions = torch.stack(ff, dim=0)
        
       

    majority_vote = torch.sum(stacked_predictions, dim=0) > (stacked_predictions.size(0) / 2)


    wee=[]
    
    majority_vote = majority_vote.int()
    wee.append(majority_vote)
    
    
 
    f1 = f1_score(  labels.cpu(),     majority_vote, average='macro')
    print(f1,"majority vote")
    
    
    num_classes=2
    conf_matrix = torch.zeros(num_classes, num_classes, dtype=torch.int64)
    for t, p in zip( labels.cpu(),   majority_vote):
        conf_matrix[t, p] += 1

    # Compute metrics
    precision, recall, accuracy_per_class, f1, overall_acc = compute_metrics(conf_matrix)
    print("Precision:", precision, "Recall:", recall, "Accuracy per class:", accuracy_per_class, "F1:", f1, "Overall Accuracy:", overall_acc)
  
    ss=torch.stack(wr, dim=0)
    aa=torch.max(ss, 0)[0] 
    
    pr=torch.max(aa, 1)[1]
    
    wee.append(pr.cpu())
    f1 = f1_score(  labels.cpu(),     pr.cpu(), average='macro')
    print(f1,"max prob.")
    
    
    
     
    aa=0
    for i in range(len(wr)):
        aa=aa+wr[i]
    
    pr=torch.max(aa, 1)[1]
    wee.append(pr.cpu())
    f1 = f1_score(  labels.cpu(),     pr.cpu(), average='macro')
    print(f1,"summ prob.")
 
    stacked_predictions = torch.stack(wee, dim=0)
    majority_vote = torch.sum(stacked_predictions, dim=0) > (stacked_predictions.size(0) / 2)
    majority_vote = majority_vote.int()
  
    

    f1 = f1_score(  labels.cpu(),     majority_vote, average='macro')
    print(f1,"avg. of all ensemble")
     
    return wr



In [ ]:
import torch.nn as n


def torch_kron(a, b):
    a_shape = [a.size(0), a.size(1)]
    b_shape = [b.size(0), b.size(1)]
    return torch.reshape(torch.reshape(a, [a_shape[0], 1, a_shape[1], 1]) * torch.reshape(b, [1, b_shape[0], 1, b_shape[1]]), [a_shape[0] * b_shape[0], a_shape[1] * b_shape[1]])



class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x


h= feature=512#1024#512#1024#50272#128256#51200#1024#768#1024#768#5376#1024#8064000#768#1024
n=2
labell = torch.arange(n, dtype=torch.int64)#.to(f'cuda:{model.device_ids[0]}') #.to(f'cuda:{x.device_ids[0]}')
p = torch.tensor(np.ones((n, 1)), dtype=torch.float32)#.to(f'cuda:{model.device_ids[0]}') #.to(f'cuda:{x.device_ids[0]}')
ones_tensor = torch.tensor(np.ones((1, n)), dtype=torch.float32)#.to(f'cuda:{model.device_ids[0]}') #.to(f'cuda:{x.device_ids[0]}')
A=np.identity(n) 
L=np.kron(A,np.ones([h,1])) 
M = torch.tensor(L, dtype=torch.float32)#.to(f'cuda:{model.device_ids[0]}') #.to(f'cuda:{x.device_ids[0]}')
    

In [ ]:
torch.cuda.empty_cache()
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False




model=T()
model.model.lm_head= Identity()


In [ ]:
loss_fn = nn.CrossEntropyLoss()
 

optimizer = torch.optim.AdamW([
    {'params': model.conv1.parameters()},
    {'params': model.encoder.parameters()},
    {'params': model.decoder.parameters()}
], lr=0.00001)




model.train()
for i in range(1201):

    
  
    optimizer.zero_grad() 
    loss=0
    for mm in range(2):
        ww=[]
        wwe=[]
        selected_indices = np.random.choice(2,2 , replace=False)
        for ii in range(2):
            # Find indices where the value is equal to 3
            indices_where_equal_to_3 = torch.nonzero( torch.tensor(y_train) == selected_indices[ii], as_tuple=False)


            # Your tensor
            your_tensor =  indices_where_equal_to_3

            # Generate random indices
            random_indices = np.random.choice( your_tensor.size(0), 2, replace=False)  #torch.randperm(your_tensor.size(0))[:2]

            # Select two random numbers
            selected_numbers = your_tensor[random_indices]




            ww.append(selected_numbers[0])
            wwe.append(selected_numbers[1])

        dd=ww+wwe
        dt = np.stack([tensor.numpy() for tensor in  dd])
        dt=dt.T



        inputss=X_train[dt][0]#.to(f'cuda:{model.device_ids[0]}') 
        _,tensor2 = model(X_train_[dt][0],inputss )

        t1=tensor2[0:n]
        t2=tensor2[n:n*2]


        z = torch.matmul(p, t1.reshape(1, n*h)) - torch_kron(ones_tensor, t2)


        logits = -torch.matmul(torch.square(z), M)

        loss =  loss+  loss_fn(logits,labell)
    loss.backward()
    optimizer.step()
    print(loss,i)
    if i%100==0 and i>0   :
        
        
        
        
         

        train_features, train_labels = extract_features(model,  train_loader ) 
        test_features, test_labels = extract_features(model,  test_loader )
        
        evall()
        model.train()
        
  